In [20]:
#!./venv/bin/python3
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import requests

from time import sleep, time
from datetime import datetime

In [24]:
access_token = 'EAACBRrCNUOwBAABVzTeRCtaGlETI3jz9n7uNEkSOZBiCPhF8I14g4VLNCZC5AQ9rxKR2b4j8j15vTF8Pi8wmiSSN8zS6Srw2q0eZAD7ZCLVW8XFySHTuxagCYArYCtrGUv6xKsBGa6un9fYMUrQ1Tfow9TZBBjKkcNZC05J1wjVP39f50pR1M3Nw2wCllrFJnQVblTBEW3MAZDZD'

fields= 'id,name,posts,groups,likes'
url = 'https://graph.facebook.com/v4.0/me?fields={fields}&access_token={access_token}'.format(fields=fields, access_token=access_token)

# url = 'https://graph.facebook.com/v4.0/{page_id}/feed/?fields={fields}&access_token={access_token}'.format(fields=fields, access_token=access_token)


In [25]:
resp = requests.get(url)
resp

<Response [200]>

In [26]:
resp.json()

{'id': '1500855749963737', 'name': '박은진'}